# 3D Tennis Shot Recognition through Spatial-Temporal Graph Convolutional Networks

Using the method described in [Learning Three Dimensional Tennis Shots UsingGraph Convolutional Networks](https://www.mdpi.com/1424-8220/20/21/6094) by M. Skublewska-Paszkowska, P. Powroznikand, E. Lukasi (2020).

The paper mentions recording both the player and the racket to identify shots. It tested putting data into the ST-GCN with and without fuzzying of data, where fuzzying returned more accurate results.
The input used is from the 3D skeleton videos from the [THETIS](http://thetis.image.ece.ntua.gr)(THree dimEnsional TennIs Shots human action dataset).

The ST-GCN is implemented with the [Spektral](https://graphneural.network/getting-started/) library.

Loading the dataset into graphs.

In [17]:
from spektral import data as spkdata
from os import listdir, mkdir
from os.path import isdir, join
from numpy import asarray, savez, load

joint_names = [
    'HEAD',
    'LEFT_ELBOW',
    'LEFT_FOOT',
    'LEFT_HAND',
    'LEFT_HIP',
    'LEFT_KNEE',
    'LEFT_SHOULDER',
    'NECK',
    'RIGHT_ELBOW',
    'RIGHT_FOOT',
    'RIGHT_HAND',
    'RIGHT_HIP',
    'RIGHT_KNEE',
    'RIGHT_SHOULDER',
    'TORSO'
    ]


class JointDataset(spkdata.Dataset):
    out = './data/skeleton_npz'
    expertise_sets = set('ONI_AMATEURS', 'ONI_EXPERTS')


    def __init__(**kwargs):
        super().__init__(**kwargs)


    def skeleton_to_matrix(data_path:str) -> list:
        joint_no = 0
        matrices = tuple([[] for _ in range(15)])
        with open(data_path) as file:
            for line in file:
                if '\n' == line: # end of file
                    break
                if 'FRAME' in line: # start filling in data for next frame
                    joint_no = 0
                    continue
                coordinates = [float(c) for c in line.split(' ')]
                matrices[joint_no].append(coordinates) # adds the coordinates for one frame
                joint_no += 1
        return asarray(matrices)


    def download(self) -> None:

        # Create output directory
        if not isdir(self.out): 
            mkdir(self.out)

        for expertise_lvl in self.expertise_sets:
            
            # Set input directory
            root: str = join('./data/THETIS_Skeletal_Joints/normal_oniFiles/', expertise_lvl) # Chooses the data without data fuzzing.
            
            # Create expertise output directory
            expertise_out: str = join('./data/skeleton_npz', expertise_lvl)
            if not isdir(expertise_out): 
                mkdir(expertise_out)

            # Gather shot lists
            shots: list = [shot_dir for shot_dir in listdir(root) if isdir(join(root, shot_dir))]
            for shot in shots:
                
                shot_root: str = join(root, shot)
                shot_out_dir: str = join(expertise_out, shot)

                if not isdir(shot_out_dir): 
                    mkdir(shot_out_dir)

                shot_files: list = [ _ for _ in listdir(shot_root)]
                for shot_file in shot_files:

                    in_path: str = join(shot_root, shot_file)
                    s_name: str = shot_file.replace('.txt', '')
                    filename: str = join(shot_out_dir, f'graph_{s_name}')

                    matrix_array = JointDataset.skeleton_to_matrix(in_path)

                    savez(
                        filename, 
                        HEAD = matrix_array[0], 
                        LEFT_ELBOW = matrix_array[1], 
                        LEFT_FOOT = matrix_array[2], 
                        LEFT_HAND = matrix_array[3], 
                        LEFT_HIP = matrix_array[4], 
                        LEFT_KNEE = matrix_array[5], 
                        LEFT_SHOULDER = matrix_array[6], 
                        NECK = matrix_array[7], 
                        RIGHT_ELBOW = matrix_array[8], 
                        RIGHT_FOOT = matrix_array[9], 
                        RIGHT_HAND = matrix_array[10], 
                        RIGHT_HIP = matrix_array[11],
                        RIGHT_KNEE = matrix_array[12], 
                        RIGHT_SHOULDER = matrix_array[13], 
                        TORSO = matrix_array[14]
                    ) # would like to avoid hardcoding this

    def read(self) -> list:
        output: list = []

        for expertise_lvl in self.expertise_sets:
            read_dir: str = join(self.out, expertise_lvl)


            for shot in [_ for _ in listdir(read_dir) if isdir(join(read_dir, _))]:

                shot_instance_dir: str = join(read_dir, shot)

                for shot_instance in [instance for instance in listdir(shot_instance_dir)]: # something wrong here? shot_instance isn't being used
                    data = load(shot_instance_dir)

                    output.append(spkdata.Graph(
                        HEAD = data['HEAD'], 
                        LEFT_ELBOW = data['LEFT_ELBOW'], 
                        LEFT_FOOT = data['LEFT_FOOT'],
                        LEFT_HAND = data['LEFT_HAND'], 
                        LEFT_HIP = data['LEFT_HIP'], 
                        LEFT_KNEE = data['LEFT_KNEE'],
                        LEFT_SHOULDER = data['LEFT_SHOULDER'], 
                        NECK = data['NECK'], 
                        RIGHT_ELBOW = data['RIGHT_ELBOW'],
                        RIGHT_FOOT = data['RIGHT_FOOT'], 
                        RIGHT_HAND = data['RIGHT_HAND'], 
                        RIGHT_HIP = data['RIGHT_HIP'],
                        RIGHT_KNEE = data['RIGHT_KNEE'], 
                        RIGHT_SHOULDER = data['RIGHT_SHOULDER'], 
                        TORSO = data['TORSO']
                    ))             

        return output


dataset = JointDataset()
dataset.download()
dataset.read()